# Sentiment Analysis on Yelp Open Dataset for Review Classification

Il notebook seguente andrà ad illustrare l'intero processo per l'implementazione di un modello di Sentiment Analysis in grado di classificare le reviews in positive o negative tramite l'uso di modelli di Deep Learning.

### Import Libraries

In [ ]:
# librerie di default
import pandas as pd
import numpy as np

# librerie per il data analysis
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
%matplotlib inline

# librerie per il text manipulation
import gensim
from gensim.parsing.preprocessing import remove_stopwords
import nltk as nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from collections import Counter, defaultdict
from datetime import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# librerie per il data modelling
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

## 1. Data Loading

Date le grandezze eccessive del dataset di input, si è deciso di caricare i dati al suo interno tramite la divisione in blocchi di grandezza pari a chunksize. Inoltre, tramite la documentazione fornita da Yelp, siamo stati in grado di tener conto del numero di byte da caricare grazie alla nota tipizzazione delle informazioni.

In [ ]:
# definiamo i tipi degli attributi JSON per l'attributo dtype di read_json
rtypes = {  "review_id": str,
            "user_id":str,
            "business_id":str,
            "stars": np.float16, 
            "useful": np.int32, 
            "funny": np.int32,
            "cool": np.int32,
            "text" : str,
           }

# file path del dataset json
path = './data/yelp_academic_dataset_review.json'

# grandezza dei chunk
chunkSize = 10000

In [ ]:
%%time
# creazione del JsonReader
review = pd.read_json(path, lines=True,
                      orient="records",
                      dtype=rtypes,
                      chunksize=chunkSize)
chunkList = []

# utilizzo della segmentazione in chunk per creare dal JsonReader il dataframe
for chunkReview in review:
    # rimozione degli attributi id
    chunkReview = chunkReview.drop(['review_id', 'user_id','business_id'], axis=1)
    chunkList.append(chunkReview)
    
# concatenazione degli elementi nella chunkList per righe
df = pd.concat(chunkList, ignore_index=True, axis=0)

In [ ]:
# visualizzazione degli elementi in testa
df.head()

## 2. Data Analysis

Durante la fase di Data Analysis, abbiamo concentrato il nostro interesse interesse sul bilanciamento delle valutazioni relative alla colonna stars, possibili correlazioni tra le colonne numeriche secondarie (cool, funny, useful) e analisi sulle frequenze di parole e lunghezze dei testi per la colonna texts.

In [ ]:
# informazioni sulle colonne del dataframe e su quante entries o righe si hanno
df.info()

### 2.1 Stars Analysis

In [ ]:
# definire la grandezza della figura
plt.figure(figsize=(8,8))

# contare i vari valori di stars e visualizzarli su un diagramma a torta
df['stars'].value_counts().plot.pie(startangle=60)

# definire il titolo del plot
plt.title('Distribuzione dei valori per l\'attributo stars')

Le quantità di recensioni, classificate in base al numero di stelle assegnate, è sbilanciata. Si ha un maggior numero per le recensioni con 5 e 4 stelle rispetto a quelle con 1, 2 o 3 stelle.

In [ ]:
# distribuzione dei valori in reviews positive e negative
binstars = pd.DataFrame()
binstars['stars'] = [0 if star <= 3.0 else 1 for star in df['stars']]
# definire la grandezza della figura
plt.figure(figsize=(8,8))


# contare i vari valori di stars e visualizzarli su un diagramma a torta
binstars['stars'].value_counts().plot.pie(startangle=60)

# definire il titolo del plot
plt.title('Distribuzione dei valori positivi e negativi')

### 2.2 Cool, Fun and Useful Analysis

In [ ]:
# Aggiunta di una feature per l'analisi della lunghezza dei testi
df['textLength']  = df['text'].str.len()

In [ ]:
# definire le correlazioni
corr = df.corr()

# generazione dell'heatmap
sns.heatmap(corr)

Non sono presenti particolari correlazioni forti tra i funny, useful e cool con i valori dati a stars o text.

### 2.3 Text Analysis

In [ ]:
%%time

# definisce un sottoinsieme delle righe del dataset
subset = df[:100000]
# concatenazione dei testi di ogni riga in una singola stringa
inputText = ' '.join(subset['text']).lower()

# creazione di un wordcloud andando ad ignorare le stopwords
wordCloud = WordCloud(background_color='white', stopwords=gensim.parsing.preprocessing.STOPWORDS).generate(inputText)
# setting della visualizzazione utilizzando una interpolazione bilineare
plt.imshow(wordCloud, interpolation='bilinear')

# rimozione degli assi
plt.axis('off')
# visualizzazione del wordcloud rappresentante le parole più usate nel testo di una recensione
plt.show()

Poichè la maggior parte delle recensioni fanno riferimento ad attività che forniscono servizi (ristorazione o di altro genere), l'utilizzo di parole che possano descrivere il luogo o i vari aspetti dell'attività sono quelle riscontrate con più frequenza.

In [ ]:
# calcolo della frequenza dei termini più utilizzati
wordTokens = word_tokenize(inputText)
tokens = list()
for word in wordTokens:
    if word.isalpha() and word not in gensim.parsing.preprocessing.STOPWORDS:
        tokens.append(word)
tokenDist = FreqDist(tokens)
# per questioni di visualizzazione, andiamo a prendere solamente i primi 20 termini utilizzati
dist = pd.DataFrame(tokenDist.most_common(20),columns=['term', 'freq'])

In [ ]:
# rappresentazione grafica dei risultati
fig = plt.figure(figsize=(14,8))
ax = fig.add_axes([0,0,1,1])
x = dist['term']
y = dist['freq']
ax.bar(x,y)
plt.title('Frequenza dei termini più utilizzati')
plt.show()

Tramite il grafico delle frequenze, possiamo notare come la maggior parte delle recensioni sono di natura culinaria, ossia una descrizione del cibo che si è ordinato. Da osservare che, dato lo sbilanciamento delle valutazioni a favore delle recensioni con valutazioni maggiore, parole usate per giudizi positivi risultano con più frequenza.

In [ ]:
df.head()

In [ ]:
# Differenziazione della lunghezza dei testi in relazione alla valutazione data a stars
graph = sns.FacetGrid(data=df,col='stars')
graph.map(plt.hist,'textLength',bins=50,color='blue')

## 3. Data Pre-processing

La procedura di data pre-processing sarà utilizzata per la divisione delle valutazioni secondo una classificazione binaria (stars <= 3 per review negativa, positiva altrimenti). Durante tale fase, ci siamo concentrati principalmente sulla manipolazione del testo, nello specifico abbiamo ridotto la diversificazione delle parole andando a rimuovere segni di punteggiatura, stopwords e forme alternative. Infine, abbiamo utilizzato un tokenizer in grado di poter effettuare una conversione in valori numerici (stemmizzazione) dato che, trattandosi di deep learning, il calcolo relativo alle funzioni di attivazione e del processing interno di una rete si basa esclusivamente su valori di natura numerica. 

### 3.1 Rimozione colonne inutilizzate e valori nulli

In [ ]:
# cancellazione delle caratteristiche cool, funny, useful e textLength poichè non hanno correlazioni con stars.
df = df.drop(['cool', 'funny', 'useful', 'textLength'], axis=1)

In [ ]:
df.head()

In [ ]:
# rimozione di possibili testi vuoti
df['text'].dropna(inplace=True)

### 3.2 Riduzione dei testi in lowercase

In [ ]:
# ridurre la forma delle parole in minuscolo
df['text'] = [review_text.lower() for review_text in df['text']]

In [ ]:
df['text'].head()

### 3.3 Polarizzazione dei labels (stars) e bilanciamento del dataset

In [ ]:
# polarizzazione delle valutazioni a stars in due categorie: 1 = positiva, 0 = negativa

# isoliamo la colonna di testo del dataframe in texts
texts =  df['text']

# andiamo ad impostare negative tutte le recensioni con 3 o meno stelle e positive quelle con 4 e 5 stelle.
stars = [0 if star <= 3.0 else 1 for star in df['stars']]

balancedTexts = [] # rappresenta la collezione di testi presi in considerazione dal dataframe di input
balancedLabels = [] # rappresenta il nuovo valore polarizzato assegnato all'entry (0,1)

# andiamo a bilanciare il dataset andando a dividere recensioni positive e negative con limite di 1.000.000 per categoria
limit = 100000  

# posizione 0 per conteggio di recensioni negative, posizione 1 per quelle positive
negPosCounts = [0, 0] 

for i in range(0,len(texts)):
    polarity = stars[i]
    if negPosCounts[polarity] < limit: # se non si è raggiunto il limite per la categoria di polarizzazione
        balancedTexts.append(texts[i])
        balancedLabels.append(stars[i])
        negPosCounts[polarity] += 1

In [ ]:
df_balanced = pd.DataFrame()
df_balanced['text'] = balancedTexts
df_balanced['labels'] = balancedLabels
df_balanced.head()

In [ ]:
# verifica del conteggio
counter = Counter(df_balanced['labels'])
print(f'Ci sono {counter[1]} recensioni positive e {counter[0]} recensioni negative')

### 3.3 Lemmatizzazione

In [ ]:
%%time
# creazione del lemmatizer
lemmatizer = WordNetLemmatizer()

# funzione per l'aggiunta del tag semantico che evidenzia il tipo di parola da dover selezionare
def word_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None


# elaborazione sui testi del dataset
texts = df_balanced['text']
df_texts = []
for text in texts:
    # tokenizzazione del text per l'aggiunta dei tag
    word_tagged = nltk.pos_tag(nltk.word_tokenize(text))
    # mapping parole:tag del testo analizzato
    map_word_tag = list(map(lambda x: (x[0], word_tagger(x[1])), word_tagged))
    # costruzione del testo lemmatizzato
    lemmatized_text = []
    for word, tag in map_word_tag:
        if tag is None:
            # elemento non tokenizzabile
            lemmatized_text.append(word)
        else:
            # lemmmatizzazione della parola in relazione al 
            # tipo di elemento
            lemmatized_text.append(lemmatizer.lemmatize(word, tag))
    # aggiunta della parola post-lemmatizzazione al testo selezionato
    lemmatized_text = " ".join(lemmatized_text)
    # aggiunta del testo nella collezione dei testi lemmatizzati
    df_texts.append(lemmatized_text)

print(texts[0] + "\n\n")
print(df_texts[0])

In [ ]:
df_balanced['text'] = df_texts

### 3.4 Rimozione delle stop words e di caratteri non alfanumerici

In [ ]:
# Stop words da rimuovere
print(gensim.parsing.preprocessing.STOPWORDS)

In [ ]:
# Rimozione delle stop words
df_texts = []
for text in df_balanced['text']:
    df_texts.append(remove_stopwords(text))

df_balanced['text'] = df_texts

# Rimozione dei caratteri non alfanumerici
df_texts = []
for text in df_balanced['text']:
    df_texts.append(''.join(ch for ch in text if ch.isalnum() or ch == ' '))

df_balanced['text'] = df_texts

In [ ]:
print(df_balanced['text'])

### 3.5 Text Tokenization

In [ ]:
%%time
# tokenizzazione del testo andando a dividere le stringhe in una lista di lemmi
df_balanced['text'] = [nltk.word_tokenize(text) for text in df_balanced['text']]

### 3.6 Preparazione Dati Vettoriali per la Fase di Modelling

In [ ]:
# definire il numero di parole da passare all'oggetto Tokenizer
# bisogna analizzare la quantità di parole che si ha nel dataframe selezionato
map_terms = dict()
for text in df_balanced['text']:
    for word in text:
        if word not in map_terms:
            map_terms[word] = 1

print(f'There are {len(map_terms)} different words') # number of words

In [ ]:
%%time
# definizione di un tokenizer delle prime 10.000 parole più utilizzate
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_balanced['text'])
# trasformazione della sequenza di lemmi in sequenze di interi in modo da valutare più velocemente le parole
sequences = tokenizer.texts_to_sequences(df_balanced['text'])
# Sequenze di massimo 200 unità. Se vi sono testi con sequenze più lunghe esse vengono troncate, altrimenti si avrà 
# un riempimenti di 0 per testi undersized.
text_sequence = pad_sequences(sequences, maxlen=200)
labels = np.array(df_balanced['labels'])

In [ ]:
# check parziale degli indici delle parole 
word_index = tokenizer.word_index
# prendiamo le prime 50 parole indicizzate
check = {key: value for key, value in word_index.items() if value <= 50}
print(check)

Il vettore dei valori numerici ha un dominio pari a 20.000 parole differenti tra le 132.062 parole totali. Si andrà, quindi, a selezionare 1/6 delle parole presenti nelle reviews che, però, ha una rilevanza maggiore rispetto ai 5/6 restanti poichè hanno occorrenze maggiori. Inoltre, la sequenza ordinata creata andrà a seguire l'ordine di occorenza dei termini all'interno dei testi di 300 parole (grandezza massima).

## 4. Modelling

In questa fase è possibile trovare modelli alternativi utilizzati oggigiorno nel campo NLP. Nello specifico, si propone una triplice alternativa che vede l'utilizzo di un modello basato su LSTM, un modello di convulational neural network che va a supporto di LSTM e, infine, un modello di LSTM bidirezionale. Possiamo affermare che tutti i modelli hanno raggiunto un livello di precisione accettabile; ciò non toglie che vi possano essere vari miglioramenti che possano incrementarne le prestazioni.

In [ ]:
# Checking sulle compile flags di tensorflow
print(tf.sysconfig.get_compile_flags())
print(tf.__version__)

In [ ]:
# Classe per calcolare F1-Score delle epoche durante la fase di training e validation dei modelli
class F1History(tf.keras.callbacks.Callback):

    def __init__(self, train, validation=None):
        super(F1History, self).__init__()
        self.validation = validation
        self.train = train

    # stampa dei valori di F1-Score alla fine di ogni epoch
    def on_epoch_end(self, epoch, logs={}):

        logs['F1_score_train'] = float('-inf')
        X_train, y_train = self.train[0], self.train[1]
        y_pred = (self.model.predict(X_train).ravel()>0.5)+0
        score = f1_score(y_train, y_pred)       

        if (self.validation):
            logs['F1_score_val'] = float('-inf')
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_val_pred = (self.model.predict(X_valid).ravel()>0.5)+0
            val_score = f1_score(y_valid, y_val_pred)
            logs['F1_score_train'] = np.round(score, 5)
            logs['F1_score_val'] = np.round(val_score, 5)
        else:
            logs['F1_score_train'] = np.round(score, 5)

In [ ]:
# train and test splitting
x_train, x_test, y_train, y_test = train_test_split(text_sequence , labels ,random_state=520, test_size=0.33, shuffle=True)

# train and validation splitting
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.33, random_state=1)

In [ ]:
print(len(x_train))
print(len(x_val))
print(len(x_test))

### 4.1 Modello basato su LSTM

In [ ]:
# creazione di un modello sequenziale vuoto in cui aggiungere i vari layers
model_lstm = keras.Sequential()

# aggiunta dei layers
model_lstm.add(layers.Embedding(10000, 128, 
                                input_length=200))
model_lstm.add(layers.LSTM(128, 
                           dropout=0.2, 
                           recurrent_dropout=0.2));
model_lstm.add(layers.Dense(1, activation='sigmoid'));

model_lstm.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

model_lstm.summary()

In [ ]:
results_lstm = model_lstm.fit(x_train, y_train, epochs=3, callbacks=[F1History(train=(x_train,y_train),validation=(x_val,y_val))])

In [ ]:
model_lstm.evaluate(x_test, y_test)

modelLSTM.evaluate(xTest, yTest)

### 4.2 Modello basato su CNN

In [ ]:
model_cnn = keras.Sequential()
model_cnn.add(layers.Embedding(10000, 128, input_length=200)) #layer iniziali 
model_cnn.add(layers.Dropout(0.25)) # layer di dropout esterno in seguito ad Embedding
model_cnn.add(layers.Conv1D(128,
                        4,
                        activation='relu'))
model_cnn.add(layers.MaxPooling1D(pool_size=4))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(128, activation='relu'))
model_cnn.add(layers.Dropout(0.25))
model_cnn.add(layers.Dense(1, activation='sigmoid'))


model_cnn.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])



model_cnn.summary()

In [ ]:
results_cnn = model_cnn.fit(x_train, y_train, epochs=3, callbacks=[F1History(train=(x_train,y_train),validation=(x_val,y_val))])

In [ ]:
model_cnn.evaluate(x_test, y_test)

### 4.1.1 Modello basato CNN + LSTM

In [ ]:
# creazione di un modello sequenziale vuoto in cui aggiungere i vari layers
model_lstm = keras.Sequential()

# aggiunta dei layers
model_lstm.add(layers.Embedding(10000, 128, input_length=200))
model_lstm.add(layers.Dropout(0.25)) # aggiunta di un layer di dropout per la regolarizzazione versoo i convulational layers
model_lstm.add(layers.Conv1D(128, 
                             4, 
                             activation='relu'))
model_lstm.add(layers.MaxPooling1D(pool_size=4))
model_lstm.add(layers.LSTM(128));
model_lstm.add(layers.Dropout(0.25)) # aggiunta di un layer di dropout che prende l'output dei layer LSTM in input
model_lstm.add(layers.Dense(1, activation='sigmoid')); 

model_lstm.compile(
    loss='binary_crossentropy', 
    optimizer='adam',
    metrics=['accuracy'])

model_lstm.summary()

In [ ]:
results_lstm = model_lstm.fit(x_train, y_train, epochs=3, callbacks=[F1History(train=(x_train,y_train),validation=(x_val,y_val))])

In [ ]:
model_lstm.evaluate(x_test, y_test)

### 4.3 Modello basato su LSTM bidirezionale

In [ ]:
model_bid = keras.Sequential()
model_bid.add(layers.Embedding(10000, 128, input_length=200))
model_bid.add(layers.Dropout(0.25))
model_bid.add(layers.Conv1D(128,
                        4,
                        activation='relu'))
model_bid.add(layers.MaxPooling1D(pool_size=4))
model_bid.add(layers.Bidirectional(layers.LSTM(128)))
model_bid.add(layers.Dropout(0.25))
model_bid.add(layers.Dense(1, activation='sigmoid'))
model_bid.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

model_bid.summary()

In [ ]:
results_bid = model_bid.fit(x_train, y_train, epochs=3, callbacks=[F1History(train=(x_train,y_train),validation=(x_val,y_val))])

In [ ]:
model_bid.evaluate(x_test,y_test)

In [ ]:
from keras.metrics import TruePositives, TrueNegatives, FalseNegatives, FalsePositives

model_bid_lstm = keras.Sequential([layers.Embedding(10000, 128, input_length=200),
                               layers.Dropout(0.25),
                               layers.Conv1D(128, 4, activation='relu'),
                               layers.MaxPooling1D(pool_size=4),
                               layers.Bidirectional(layers.LSTM(64, return_sequences = True)),
                               layers.LSTM(32, recurrent_dropout=0.4),
                               layers.Dropout(0.25),
                               layers.Dense(1, activation='sigmoid')])
model_bid_lstm.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])




model_bid_lstm.summary()


In [ ]:
results_bid_lstm = model_bid_lstm.fit(x_train, y_train, epochs=3, callbacks=[F1History(train=(x_train,y_train),validation=(x_val,y_val))])

## 5. Save Models

Data l'eccessiva tempo speso per l'addestramento delle reti neurali proposte, abbiamo deciso di salvare tramite la libreria pickle le componenti e i modelli addestrati in modo da poterli rendere disponibili per l'analisi delle prestazioni e l'utilizzo.

In [ ]:
import pickle

# salviamo il tokenizer e i modelli su file
with open("dump/keras_tokenizer.pickle", "wb") as f:
    pickle.dump(tokenizer, f)
    
model_lstm.save("dump/model/yelp_model_lstm.hdf5")
model_cnn.save("dump/model/yelp_model_cnn.hdf5")
model_bid.save("dump/model/yelp_bidirectional_lstm.hdf5")
model_bid_lstm.save("dump/model/yelp_combination_bid_lstm.hdf5")


In [ ]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import pickle

# carichiamo il tokenizer e il modello da file
with open("dump/keras_tokenizer.pickle", "rb") as f:
    tokenizer = pickle.load(f)

# TODO: load other models
model_lstm = load_model("dump/model/yelp_model_lstm.hdf5")
model_cnn = load_model("dump/model/yelp_model_cnn.hdf5")
model_bid = load_model("dump/model/yelp_bidirectional_lstm.hdf5")
model_bid_lstm = load_model("dump/model/yelp_combination_bid_lstm.hdf5")

# definiamo gli esempi su cui testare il modello
examples_reviews = ["slow orders but good food", "Delicious foods! Awesome!", "Bad food, bad people... horrible!"]

# usiamo il tokenizer per creare sequenze di interi da dare al modello
sequences = tokenizer.texts_to_sequences(examples_reviews)
data_examples = pad_sequences(sequences, maxlen=200)

# effettuare le predizioni e stampare i risultati
predictions_lstm = model_lstm.predict(data_examples)
predictions_cnn = model_cnn.predict(data_examples)
predictions_bid = model_bid.predict(data_examples)
predictions_bid_lstm = model_bid_lstm.predict(data_examples)

print(f"Risultati modello LSTM:\n {predictions_lstm}\n\n"+
    f"Risultati modello CNN:\n {predictions_cnn}\n\n" + 
      f"Risultati modello biLSTM:\n {predictions_bid}\n\n" +
     f"Risultati modello combinato biLSTM + LSTM:\n {predictions_bid_lstm}")

In [ ]:
# grafico di paragone per i valori della training accuracy tra i modelli proposti
plt.plot(results_bid.history['accuracy'])
plt.plot(results_cnn.history['accuracy'])
plt.plot(results_lstm.history['accuracy'])
plt.plot(results_bid_lstm.history['accuracy'])
plt.title('Analisi training accuracy dei modelli')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['biLSTM', 'CNN', 'LSTM', 'biLSTM+LSTM'], loc='best')
plt.show()


# grafico di paragone per i valori della loss function tra i modelli proposti
plt.plot(results_bid.history['loss'])
plt.plot(results_cnn.history['loss'])
plt.plot(results_lstm.history['loss'])
plt.plot(results_bid_lstm.history['loss'])
plt.title('Analisi valori loss dei modelli')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['biLSTM', 'CNN', 'LSTM', 'biLSTM+LSTM'], loc='best')
plt.show()

# grafico di paragone per i valori dell'accuracy in validazione
plt.plot(results_bid.history['val_accuracy'])
plt.plot(results_cnn.history['val_accuracy'])
plt.plot(results_lstm.history['val_accuracy'])
plt.plot(results_bid_lstm.history['val_accuracy'])
plt.title('Analisi validation accuracy dei modelli')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['biLSTM', 'CNN', 'LSTM', 'biLSTM+LSTM'], loc='best')
plt.show()

In [ ]:
results_cnn.history

# grafico di paragone per i valori del F1-Score in training
plt.plot(results_bid.history['F1_score_train'])
plt.plot(results_cnn.history['F1_score_train'])
plt.plot(results_lstm.history['F1_score_train'])
plt.plot(results_bid_lstm.history['F1_score_train'])
plt.title('Analisi F1 Score sul training set dei modelli')
plt.ylabel('F1 Score')
plt.xlabel('epoch')
plt.legend(['biLSTM', 'CNN', 'LSTM', 'biLSTM+LSTM'], loc='best')
plt.show()

# grafico di paragone per i valori del F1-Score in validation
plt.plot(results_bid.history['F1_score_val'])
plt.plot(results_cnn.history['F1_score_val'])
plt.plot(results_lstm.history['F1_score_val'])
plt.plot(results_bid_lstm.history['F1_score_val'])
plt.title('Analisi F1 Score sul validation set nei modelli')
plt.ylabel('F1 Score')
plt.xlabel('epoch')
plt.legend(['biLSTM', 'CNN', 'LSTM', 'biLSTM+LSTM'], loc='best')
plt.show()